## Download Data from Google-Drive

In [ ]:
'''
File Name : Data
File Link : https://drive.google.com/file/d/1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw/view?usp=share_link
File Id : '1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw'

'''
!gdown --id 1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw
!unzip ccle_ctrpv2_gdse.zip

## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import mean_squared_error , mean_absolute_error
from time import perf_counter, sleep
import math
import scipy.stats as stats
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

## Read train and test data

In [2]:
# x_train
ccle_ctrpv2 = pd.read_csv("../../data/ccle_ctrpv2_rnaseq_tpm.csv")
# x_test
ccle_ctrpv2_aac = pd.read_csv("../../data/ccle_ctrpv2_aac.csv")
# y_train
gdse_rnaseq = pd.read_csv("../../data/gdse_rnaseq_tpm.csv")
# y_test
gdse_aac = pd.read_csv("../../data/gdse_aac.csv")


## Cleaning the dataframes

In [3]:
drug_names = gdse_aac.columns.values.tolist()

ccle_ctrpv2 = ccle_ctrpv2.rename(columns={"Unnamed: 0": "sample"})
gdse_rnaseq = gdse_rnaseq.rename(columns={"Unnamed: 0": "sample"})
ccle_ctrpv2_aac = ccle_ctrpv2_aac.rename(columns={"Unnamed: 0": "sample"})
gdse_aac = gdse_aac.rename(columns={"Unnamed: 0": "sample"})

In [4]:
drug_names = drug_names[1:]

In [5]:
def remove_nan(data):
    data = data.dropna()
    data.reset_index(drop = True)
    return data

# Feature selection method

    Genetic algorithm
    
**Install**:     `!pip install feature-selection-ga`

# Training procedure

*For each drug in the drug list:*
   1. Split the data in to train and test sets, and then remove the Nan values.
   2. Merging X_train with ytrain, and X_test with  y_test because the samples in each are different.
   3. Normalize the training X. </br>
   
   *Running the following 6 times:*
   
       4. Applying 4-fold Cross Valisation
       5. Applying the relevant features selection technique, and extract the selected features.
       6. Train the model based on the CV data and extracted features
       7. Saving the best model performed in CV and the best features
       
   8. Predicting the test set using the best model from the previous section
   9. Calculating the correlation score between the predicted drug_respons and the actual drug_response

In [10]:
pearson_corr = []
kendalltau_corr = []
spearmanr_corr = []
MSE_metric = []
RMSE_metric = []
MAE_metric = []
pearson_corr_train = []
kendalltau_corr_train = []
spearmanr_corr_train = []
MSE_metric_train = []
RMSE_metric_train = []
MAE_metric_train = []
Sscaler = StandardScaler()
start = perf_counter()

for idx, drug in enumerate(drug_names):
    
    selected_features = []


    '''
        train test split
    '''
    y_train = ccle_ctrpv2_aac[['sample', drug]]
    y_test  = gdse_aac[['sample', drug]]

    y_train = remove_nan(y_train)
    y_test  = remove_nan(y_test)

    ccle_ctrpv2 = remove_nan(ccle_ctrpv2)
    gdse_rnaseq = remove_nan(gdse_rnaseq)


    '''

    Merge
    '''

    gdse_rnaseq_merged_df = pd.merge(gdse_rnaseq, y_test, on='sample', how='inner')
    ccle_ctrpv2_merged_df = pd.merge(ccle_ctrpv2, y_train, on='sample', how='inner')

    X_train = ccle_ctrpv2_merged_df.iloc[:, 1: len(gdse_rnaseq.columns)]
    X_test  = gdse_rnaseq_merged_df.iloc[:, 1: len(gdse_rnaseq.columns)]

    Y_train = ccle_ctrpv2_merged_df.iloc[:, len(gdse_rnaseq.columns):]
    Y_test  = gdse_rnaseq_merged_df.iloc[:, len(gdse_rnaseq.columns):]

    X_train_norm = Sscaler.fit_transform(X_train)
    X_train_normalized = pd.DataFrame(X_train_norm, columns=X_train.columns)

    X_test_norm = Sscaler.fit_transform(X_test)
    X_test_normalized = pd.DataFrame(X_test_norm, columns=X_test.columns)

    X_train_array = np.asarray(X_train_normalized)
    X_test_array = np.asarray(X_test_normalized)
    Y_train_array = np.asarray(Y_train).ravel()
    Y_test_array = np.asarray(Y_test).ravel()
    

    estimators = RandomForestRegressor(max_depth= 2 , n_estimators=20)

    # Define the number of folds for cross-validation
    num_folds = 4

    # Create a K-fold cross-validation object
    kfold = KFold(n_splits=num_folds)
 
    e = np.random.uniform(0, 0.2, size=(len(X_train_array), 100))

    X = np.hstack((X_train_array, e))
    Y = Y_train_array
    estimators.fit(X, Y)
    
    
    # Perform cross-validation
    cv_scores = cross_val_score(estimators, X, Y , cv=kfold, scoring='neg_mean_squared_error' , n_jobs=-1)
    
    # Calculate the mean squared error (MSE) scores
    mse_scores = -cv_scores
    
    # Get the best model based on cross-validation
    mymin = np.min(mse_scores)
    best_model_index = [i for i, x in enumerate(mse_scores) if x == mymin]
    #best_model_index = np.argmax(mse_scores)
    best_model = estimators
    # Retrieve the best model based on the index
    best_model = best_model.estimators_[best_model_index.pop()]

    selectors = GeneticSelectionCV(best_model,
                                  cv=4,
                                  verbose=2,
                                  scoring="neg_mean_squared_error",
                                  max_features=1000,
                                  n_population=50,
                                  crossover_proba=0.1,
                                  mutation_proba=0.1,
                                  n_generations=3,
                                  n_jobs=-1)
    
    selectors = selectors.fit(X, Y)
    selected_features = np.where(selectors.support_)[0]  # Get the indices of selected features
    selected_features = selected_features[:100] 
    
    
    X_train_new = X_train_normalized.iloc[:,selected_features]
    X_split, X_val, Y_split, y_val = train_test_split(X_train_new, Y_train, test_size=0.25, random_state=1)
    
    best_model.fit(X_split, Y_split)
    
    #Calculate metrics for the validation data
    
    predictions_train = best_model.predict(X_val)

    mse_train = mean_squared_error(np.asarray(y_val).ravel(), predictions_train)
    rmse_train = math.sqrt(mse_train)
    mae_train = mean_absolute_error(np.asarray(y_val).ravel(), predictions_train)

    # Calculate Correlation for the validation data
    spearmanr_correlation_train, _ = stats.spearmanr(np.asarray(y_val).ravel(), predictions_train)
    kendalltau_correlation_train, _ = stats.kendalltau(np.asarray(y_val).ravel(), predictions_train)
    pearson_correlation_train, _ = pearsonr(np.asarray(y_val).ravel(), predictions_train)


    pearson_corr_train.append(pearson_correlation_train)
    kendalltau_corr_train.append(kendalltau_correlation_train)
    spearmanr_corr_train.append(spearmanr_correlation_train)
    MSE_metric_train.append(mse_train)
    RMSE_metric_train.append(rmse_train)
    MAE_metric_train.append(mae_train)
    
    #Predict X test
    X_test_new = X_test_normalized.iloc[:,selected_features]
    predictions = best_model.predict(X_test_new)
    
    #Calculating Metrics for the Testing Phase
    mse = mean_squared_error(Y_test_array, predictions)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(Y_test_array, predictions)
    
    #Calculating Correlation for the Testing Phase
    
    spearmanr_correlation, _ = stats.spearmanr(Y_test_array.flatten(),  predictions.flatten())
    kendalltau_correlation, _ = stats.kendalltau(Y_test_array.flatten(),  predictions.flatten())
    pearson_correlation, _ = pearsonr(Y_test_array.flatten(),  predictions.flatten())
    
    pearson_corr.append(pearson_correlation)
    kendalltau_corr.append(kendalltau_correlation)
    spearmanr_corr.append(spearmanr_correlation)
    MSE_metric.append(mse)
    RMSE_metric.append(rmse)
    MAE_metric.append(mae)
    
end = perf_counter()    


Selecting features with genetic algorithm.
gen	nevals	avg                               	std                                  	min                         	max                               
0  	50    	[ -0.005374 554.32       0.000715]	[   0.000246  248.150474    0.000223]	[-0.00591 25.       0.00033]	[ -0.00474  999.         0.001299]
1  	8     	[-600.004872  574.86      600.000592]	[ 2374.867187   322.221663  2374.868268]	[-10000.          130.            0.000405]	[   -0.00474  1579.      10000.     ]
2  	11    	[-1400.004247   653.14      1400.000504]	[ 3469.868601   415.653173  3469.870111]	[-10000.          130.            0.000405]	[   -0.00474  1839.      10000.     ]
3  	7     	[-1400.00414    644.98      1400.000488]	[ 3469.868644   413.24581   3469.870118]	[-10000.         130.           0.00043]   	[   -0.00474  1759.      10000.     ]
Selecting features with genetic algorithm.
gen	nevals	avg                               	std                                  	min         

1  	14    	[-1200.003729   776.82      1200.00079 ]	[ 3249.613985   411.804356  3249.61507 ]	[-10000.          120.            0.000321]	[   -0.003518  1854.       10000.      ]
2  	8     	[-1200.003469   731.8       1200.000737]	[ 3249.614081   441.085615  3249.61509 ]	[-10000.          267.            0.000325]	[   -0.003518  1920.       10000.      ]
3  	8     	[-800.00348   647.12      800.000804]   	[ 2712.930967   356.611982  2712.931756]	[-10000.          267.            0.000333]	[   -0.003518  1361.       10000.      ]
Selecting features with genetic algorithm.
gen	nevals	avg                               	std                                  	min                            	max                               
0  	50    	[ -0.013398 511.88       0.000766]	[   0.000523  275.609988    0.000347]	[-0.014561 19.        0.000135]	[ -0.012167 977.         0.001704]
1  	6     	[-1200.011393   599.38      1200.000542]	[ 3249.611155   387.029296  3249.615162]	[-10000.           19.      

3  	8     	[-1200.007065   901.34      1200.000412]	[ 3249.612753   315.367317  3249.61521 ]	[-10000.          534.            0.000346]	[   -0.007981  1820.       10000.      ]
Selecting features with genetic algorithm.
gen	nevals	avg                               	std                                  	min                            	max                               
0  	50    	[ -0.00693  555.16       0.001206]	[   0.000587  302.673181    0.00063 ]	[-0.008366 37.        0.000202]	[ -0.005709 985.         0.00316 ]
1  	18    	[-1800.005244   607.34      1800.001   ]	[ 3841.872086   443.375354  3841.874074]	[-10000.           37.            0.000439]	[   -0.005709  1851.       10000.      ]
2  	4     	[-400.005879  495.16      400.001166]   	[ 1959.590594   368.712211  1959.591556]	[-10000.           89.            0.000439]	[   -0.005709  1249.       10000.      ]
3  	11    	[-600.005563  692.64      600.001223]   	[ 2374.867012   420.11364   2374.868108]	[-10000.          89.       

1  	3     	[-600.032904  592.82      600.003759]	[ 2374.860104   333.247397  2374.867468]	[-10000.           80.            0.002191]	[   -0.033406  1754.       10000.      ]
2  	7     	[-800.031562  652.34      800.003044]	[ 2712.922686   384.910723  2712.931096]	[-10000.           80.            0.002549]	[   -0.033406  1952.       10000.      ]
3  	9     	[-1200.029703   695.68      1200.002784]	[ 3249.604393   326.166488  3249.614334]	[-10000.          404.            0.002942]	[   -0.032987  1879.       10000.      ]
Selecting features with genetic algorithm.
gen	nevals	avg                               	std                                  	min                            	max                               
0  	50    	[ -0.041652 485.96       0.005141]	[   0.001698  295.569685    0.0015  ]	[-0.046099 14.        0.002081]	[ -0.038081 997.         0.009179]
1  	7     	[-1000.036021   574.46      1000.004544]	[ 2999.987993   442.338613  2999.998485]	[-10000.           33.            

3  	12    	[-1600.009464   531.54      1600.000917]	[ 3666.056425   434.984469  3666.060156]	[-10000.           45.            0.000521]	[   -0.010793  1740.       10000.      ]


In [11]:
execution_time = end - start

df = pd.DataFrame({"Drugs": drug_names, "pearsonCor": pearson_corr , "spearmanCor": spearmanr_corr , "kendallCor": kendalltau_corr , 
                  "RMSE": RMSE_metric ,  "MSE": MSE_metric , "MAE": MAE_metric , "Time": execution_time})
df.to_csv('RF-results/Result_Genetic_RandomForrest_test.csv', index=False)

df = pd.DataFrame({"Drugs": drug_names, "pearsonCor": pearson_corr_train , "spearmanCor": spearmanr_corr_train , "kendallCor": kendalltau_corr_train , 
                  "RMSE": RMSE_metric_train ,  "MSE": MSE_metric_train , "MAE": MAE_metric_train , "Time": execution_time})
df.to_csv('RF-results/Result_Genetic_RandomForrest_train.csv', index=False)

In [139]:
np.mean(correlatoins)

0.2978840247375762

In [164]:
len(correlatoins)

22